In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [ ]:
data_train= pd.read_csv('/content/drive/MyDrive/DoAn/data/train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/DoAn/data/test.csv')

In [ ]:
save_path = '/content/drive/MyDrive/DoAn/models'

In [ ]:
data_train.head(1)

,Hotel_Address,Average_Score,Hotel_Name,Negative_Review,Positive_Review,Reviewer_Score,Tags,Hotel_Nation,Negative_Review_Nouns,Positive_Review_Nouns,Hotel_Location,Hotel_Service,Hotel_Room,Hotel_F&A,Hotel_Meal,Customer_Location,Customer_Service,Customer_Room,Customer_F&A,Customer_Meal
0,11 Cadogan Gardens Sloane Square Kensington an...,8.7,11 Cadogan Gardens,empty,The staff are so friendly and helpful,10.0,"[' Leisure trip ', ' Solo traveler ', ' Superi...",UK,empty,staff,0.394052,0.401487,-0.107807,-0.05948,0.037175,0.0,1.0,0.0,0.0,0.0


In [ ]:
X_train = data_train[['Average_Score', 'Hotel_Location', 'Hotel_Service', 'Hotel_Room', 'Hotel_F&A', 'Hotel_Meal',
              'Customer_Location', 'Customer_Service', 'Customer_Room', 'Customer_F&A', 'Customer_Meal']]
y_train = data_train['Reviewer_Score']

In [ ]:
X_test = data_test[['Average_Score', 'Hotel_Location', 'Hotel_Service', 'Hotel_Room', 'Hotel_F&A', 'Hotel_Meal',
              'Customer_Location', 'Customer_Service', 'Customer_Room', 'Customer_F&A', 'Customer_Meal']]
y_test = data_test['Reviewer_Score']

#LINEAR REGRESSION

In [ ]:
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)
y_pred = model_linear.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

Mean Squared Error: 2.187180019232408
Mean Abosolute Error: 1.1474863146122622


# RANDOM FOREST REGRESSION

In [ ]:
model_rdr = RandomForestRegressor(n_estimators=10,max_depth = 10, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#odel_file_path = os.path.join(save_path, 'rdr_10_10_42.joblib')
#joblib.dump(model_rdr, model_file_path)
p#rint(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.0832774859675434
Mean Abosolute Error: 1.1118995598940986
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_10_10_42.joblib


In [ ]:
model_rdr = RandomForestRegressor(n_estimators=10,max_depth = 20, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#model_file_path = os.path.join(save_path, 'rdr_10_20_42.joblib')
#joblib.dump(model_rdr, model_file_path)
#print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.1970707878146456
Mean Abosolute Error: 1.1399420779879192
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_10_20_42.joblib


In [ ]:
model_rdr = RandomForestRegressor(n_estimators=100,max_depth = 10, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#model_file_path = os.path.join(save_path, 'rdr_100_10_42.joblib')
#joblib.dump(model_rdr, model_file_path)
#print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.079922340955446
Mean Abosolute Error: 1.1111954122351093
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_100_10_42.joblib


In [ ]:
model_rdr = RandomForestRegressor(n_estimators=100,max_depth = 20, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#model_file_path = os.path.join(save_path, 'rdr_100_20_42.joblib')
#joblib.dump(model_rdr, model_file_path)
#print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.167007009579471
Mean Abosolute Error: 1.1326365086161276
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_100_20_42.joblib


In [ ]:
model_rdr = RandomForestRegressor(n_estimators=100,max_depth = None, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#model_file_path = os.path.join(save_path, 'rdr_100_N_42.joblib')
#joblib.dump(model_rdr, model_file_path)
#print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.263028980996664
Mean Abosolute Error: 1.15837029490987
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_100_N_42.joblib


In [ ]:
model_rdr = RandomForestRegressor(n_estimators=50,max_depth = None, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#model_file_path = os.path.join(save_path, 'rdr_50_N_42.joblib')
#joblib.dump(model_rdr, model_file_path)
#print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.268700277187472
Mean Abosolute Error: 1.1598255290925488
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_50_N_42.joblib


In [ ]:
model_rdr = RandomForestRegressor(n_estimators=10,max_depth = None, random_state=42)
model_rdr.fit(X_train, y_train)

y_pred = model_rdr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

#model_file_path = os.path.join(save_path, 'rdr_10_N_42.joblib')
#joblib.dump(model_rdr, model_file_path)
#print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.306298893256962
Mean Abosolute Error: 1.16932231766137
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/rdr_10_N_42.joblib


# XGBOOST

In [ ]:
# Huấn luyện mô hình với giá trị tối ưu
model_xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4)
model_xgb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra và tính MSE
y_pred = model_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

model_file_path = os.path.join(save_path, 'xgb_100_01_4.joblib')
joblib.dump(model_xgb, model_file_path)
print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.0785664213360433
Mean Abosolute Error: 1.1110674869931225
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/xgb_100_01_4.joblib


In [ ]:
# Huấn luyện mô hình với giá trị tối ưu
model_xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
model_xgb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra và tính MSE
y_pred = model_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

model_file_path = os.path.join(save_path, 'xgb_100_01_5.joblib')
joblib.dump(model_xgb, model_file_path)
print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.0777485047389344
Mean Abosolute Error: 1.1106430537318563
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/xgb_100_01_5.joblib


In [ ]:
# Huấn luyện mô hình với giá trị tối ưu
model_xgb = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=4)
model_xgb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra và tính MSE
y_pred = model_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

model_file_path = os.path.join(save_path, 'xgb_200_01_4.joblib')
joblib.dump(model_xgb, model_file_path)
print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.0772073915346807
Mean Abosolute Error: 1.1104372861616603
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/xgb_200_01_4.joblib


In [ ]:
# Huấn luyện mô hình với giá trị tối ưu
model_xgb = xgb.XGBRegressor(n_estimators=400, learning_rate=0.1, max_depth=5)
model_xgb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra và tính MSE
y_pred = model_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

model_file_path = os.path.join(save_path, 'xgb_400_01_5.joblib')
joblib.dump(model_xgb, model_file_path)
print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.0778638402441243
Mean Abosolute Error: 1.1103567570052384
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/xgb_400_01_5.joblib


In [ ]:
# Huấn luyện mô hình với giá trị tối ưu
model_xgb = xgb.XGBRegressor(n_estimators=600, learning_rate=0.1, max_depth=4)
model_xgb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra và tính MSE
y_pred = model_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

model_file_path = os.path.join(save_path, 'xgb_600_01_4.joblib')
joblib.dump(model_xgb, model_file_path)
print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.076868205386565
Mean Abosolute Error: 1.109998134918615
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/xgb_600_01_4.joblib


In [ ]:
# Huấn luyện mô hình với giá trị tối ưu
model_xgb = xgb.XGBRegressor(n_estimators=600, learning_rate=0.1, max_depth=10)
model_xgb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra và tính MSE
y_pred = model_xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae =  mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Abosolute Error: {mae}')

model_file_path = os.path.join(save_path, 'xgb_600_01_10.joblib')
joblib.dump(model_xgb, model_file_path)
print(f'Mô hình đã được lưu vào: {model_file_path}')

Mean Squared Error: 2.1616792627316466
Mean Abosolute Error: 1.1309549269009425
Mô hình đã được lưu vào: /content/drive/MyDrive/DoAn/models/xgb_600_01_10.joblib
